In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import requests
import time
import json

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client.test

In [3]:
def temp_store_memory(agent_id, agents_involved, content):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "content": content, #!!!
    }
    db.agent_memories.insert_one(memory)

In [4]:
temp_store_memory("agent_001",
                  ["agent_002"],
                  "I am cooking pasta.")

In [5]:
def temp_retrieve_memories(agent_id):
    memories = db.agent_memories.find({
        "agent_id": agent_id
    }).sort("timestamp", -1)
    return list(memories)

def temp_retrieve_memories_concerning(agent_id):
    memories = db.agent_memories.find({
        "$or": [
            {"agent_id": agent_id},
            {"agents_involved": agent_id}
        ]
    }).sort("timestamp", -1)
    return list(memories)

print(temp_retrieve_memories("agent_001"))

[{'_id': ObjectId('6753376a5b47454d57030af9'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 18, 42, 2, 797000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('6732045b05fe2d87a97ac97a'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 11, 11, 14, 19, 23, 344000), 'content': 'Had this conversation with Eliana: Here\'s the summary in a way that I can easily recall:\n\n**Eliana:** "Hey Sami! Remember how we talked about trying new recipes and learning from each other?"\n\n**Sami:** "Yeah... One thing that comes to mind is when we were discussing Gordon Ramsay\'s pasta dishes. He was so enthusiastic about making them, and it was really inspiring to see him experiment with different ingredients."\n\n**Eliana:** "That sounds awesome! I loved hearing your passion for cooking, Sami. And I have a similar memory - we talked about Italian cuisine and you were excited to learn more about

In [6]:
class Agent:
    def __init__(self, name, agent_id, user_input, gender):
        self.name = name
        self.agent_id = agent_id 
        self.user_input = user_input
        self.gender = gender
        self.opinions = {}
        
    def __str__(self):
        return f"Agent {self.name} (ID: {self.agent_id}) (Description: {self.user_input}) (Opinions: {self.opinions})"
    
    def name(self):
        return self.name
    
    def agent_id(self):
        return self.agent_id
    
    def user_input(self):
        return self.user_input
    
    def gender(self):
        return self.gender

In [29]:
Sami = Agent("Sami", "agent_001", "Sami is a man who likes horses and love to go hiking. He also loves fishing. He has an opinion on everything and is kind most of the time.", "male")

Eliana = Agent("Eliana", "agent_002", "Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about ducks. She is the sweetest person on earth and would never lie or attack someone.", "female")

Alice = Agent("Alice", "agent_003", "Alice is a woman who loves cats and crochet. She doesn't like cooking at all. She is nice to people unless they are unkind to her first.", "female")

Sami.opinions["Eliana"] = "He likes Eliana and thinks her cooking advices are great"
Sami.opinions["Alice"] = "He enjoys having conversations with her about his hobbies"

Eliana.opinions["Sami"] = "She thinks Sami is cool and she likes his cooking"
Eliana.opinions["Alice"] = "She's happy to have found someone who likes animals as much as she does"

Alice.opinions["Sami"] = "She likes hearing him talk about his hiking adventures"
Alice.opinions["Eliana"] = "She likes that she's a kind and calm person"

print(Sami)
print(Eliana)
print(Alice)

Agent Sami (ID: agent_001) (Description: Sami is a man who likes horses and love to go hiking. He also loves fishing. He has an opinion on everything and is kind most of the time.) (Opinions: {'Eliana': 'He likes Eliana and thinks her cooking advices are great', 'Alice': 'He enjoys having conversations with her about his hobbies'})
Agent Eliana (ID: agent_002) (Description: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about ducks. She is the sweetest person on earth and would never lie or attack someone.) (Opinions: {'Sami': 'She thinks Sami is cool and she likes his cooking', 'Alice': "She's happy to have found someone who likes animals as much as she does"})
Agent Alice (ID: agent_003) (Description: Alice is a woman who loves cats and crochet. She doesn't like cooking at all. She is nice to people unless they are unkind to her first.) (Opinions: {'Sami': 'She likes hearing him talk about his hiking adventures', 'Eliana': "She likes that she's a kind an

In [36]:
def make_initial_prompt(agent1, agent2, subject, location):
    message_content = f"""
            Context:
            Here is a description of {agent1.name}: {agent1.user_input}.
            Here is a description of {agent2.name}: {agent2.user_input}.
            Here is what {agent1.name} thinks about {agent2.name}:
            {agent1.opinions[agent2.name]}
            Here is what {agent2.name} thinks about {agent1.name}:
            {agent2.opinions[agent1.name]}
            The context of this conversation is: {subject}
            Start directly by a quick sentence describing the scene{'' if location==None else "which is taking place at "+location}, what the agents were doing before starting the conversation and their main emotion, without an introduction.
            Then, create the conversation they had:

            {agent1.name}: "
        """
    return message_content

def make_subject(memory):
    message_content = f"""
            Context:
            This is the memory of a conversation between 2 people:
            {memory}
            Make a summary in 1 sentence or 2 with the important points and details of the conversation (for example, if they are talking about doing an activity or a dish together, keep it in memory), ignorig the first sentence.
            Start directly with the summary, not with a phrase like "here is a summary".
        """
    return message_content

def extract_emotion(memory, agent_concerned, other):
    message_content = f"""
    Agent1: {agent_concerned.name}, {agent_concerned.gender}, {agent_concerned.user_input}
    Agent2: {other.name}, {other.gender}, {other.user_input}
    Conversation: "{memory}"
    Possible emotions: ["happy", "sad", "angry", "neutral", "fearful"]
    Importance: On a scale of 1 to 10, 1 being very unimportant and mundane like a talk about the weather, 10 being very important like a breakup.

    Analyze the information about the agents and their conversation, and identify the primary emotion felt by Agent1 as well as the importance of the conversation as a result of this interaction.
    """ + """
    The response MUST be in the JSON format: {"emotion": "emotion", "importance": "importance"}. Here are a few examples of the format of the output:
    {"emotion": "happy", "importance": "10"}
    {"emotion": "sad", "importance": "5"}
    {"emotion": "neutral", "importance": "1"}
    {"emotion": "fearful", "importance": "3"}
    """
    return message_content

#The response MUST be a SINGLE word from the provided list of emotions, only from the provided list, nothing more.

def update_opinion(memory, agent_concerned, other, importance=0):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Conversation: "{memory}"

    Here is the conversation that happened between {agent_concerned.name} and {other.name}.
    The importance of this conversation to the agent is {importance} on a scale of 1 to 10, 1 being very unimportant and mundane like a talk about the weather, 10 being very important like a breakup. You should take this into account to adjust how much you update their opinion.
    Summarize what {agent_concerned.name} thought about {other.name} in one short sentence. The sentence needs to be in third person:
    """
    return message_content

def update_opinion2(opinion, agent_concerned, other, emotion=None, importance=0):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Last opinion of {agent_concerned.name} about {other.name}: {agent_concerned.opinions[other.name]}
    Analyzed phrase or conversation: "{opinion}"
    {f'Emotion felt by {agent_concerned.name}: "{emotion}"' if emotion else ''}

    Based on the information about both agents and the provided text, update the opinion of {agent_concerned.name} about {other.name}.
    If an emotion is specified, take it into account to adjust the tone or content of this opinion.
    The importance of this conversation to the agent is {importance} on a scale of 1 to 10, 1 being very unimportant and mundane like a talk about the weather, 10 being very important like a breakup. You should take this into account to adjust how much you update their opinion.
    If a part of the old opinion is correct, keep it in the new opinion.
    Juste give the new opinion, nothing else.
    """
    return message_content

def extract_location(memory):
    message_content = f"""
    Conversation: "{memory}"
    
    Based on the context of the conversation (the first sentence), extract the location of the interaction, in a single word.
    """
    return message_content

In [37]:
url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}

def chat(agent1, agent2, subject, use_memory = True, use_location = None):
    # First agent's turn
    memories = temp_retrieve_memories(agent1.agent_id)
    if (len(memories) > 0 and use_memory):
        subject_prompt = make_subject(memories[0])
        data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": subject_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }
        response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
        subject = "Here is the resume of the past conversation: " + response + " They meet some time later"+ {'' if use_location==None else "at "+use_location}
        print("Subject :", subject)
    
    message_prompt = make_initial_prompt(agent1, agent2, subject, use_location)

    data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": message_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }

    response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
    print(response)
    #temp_store_memory(agent1.agent_id, [agent2.agent_id], str(response))
    #temp_store_memory(agent2.agent_id, [agent1.agent_id], str(response))
    return response

In [39]:
dialog = chat(Eliana, Alice, "They are fighting", False, "Equestria, the land of ponies")

The sun was setting over the rolling hills of Equestria, casting a warm orange glow over the lush green grass as Eliana and Alice stood facing each other in the middle of a bustling marketplace. They were both holding baskets filled with various animal-related items, their faces red with frustration.

Eliana: "You're being completely ridiculous, Alice! You can't just take all my duck books and claim they're yours!"

Alice: "I did not take them, Eliana. You know I'm a cat person, remember? Those duck books are clearly not of any interest to me."

Eliana: "That's not the point! The point is that you were helping me with the duck exhibit at the Equestrian Museum and now you're just taking all my research and notes?"

Alice: "I was trying to help, but I thought maybe we could share the workload. You never even offered to lend them to me."

Eliana: "We agreed on a plan, Alice! We were going to work together on the exhibit, not you just swooping in and taking all my hard work."

Alice: "I'm 

In [40]:
location_prompt = extract_location(dialog)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": location_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
location = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("location:", location)

location: Equestria


In [41]:
subject_prompt = extract_emotion(dialog, Alice, Eliana)
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
emotion = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("emotion:", emotion)
emotion_json = json.loads(emotion[emotion.find("{"):emotion.rfind("}")+1])
print("\nParsed emotion:", emotion_json)

subject_prompt = update_opinion(dialog, agent_concerned=Alice, other=Eliana, importance=emotion_json["importance"])
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
opinion = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("\nOpinion about the conversation:", opinion)

subject_prompt = update_opinion2(opinion, agent_concerned=Alice, other=Eliana, emotion=emotion, importance=emotion_json["importance"])
data = {
    "model": "llama-3.2-3b-instruct",
    "messages": [
        {"role": "system", "content": ""},
        {"role": "user", "content": subject_prompt}
    ],
    "temperature": 0.7,
    "max_tokens": -1,
    "stream": False
    }
response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
print("\nnew opinion:", response)

emotion: {"emotion": "angry", "importance": "8"}

Here's the reasoning:

* Agent1 (Alice) feels angry throughout the conversation, especially when Eliana accuses her of being untrustworthy and unreasonable. The tone of Alice's responses becomes increasingly defensive and assertive as the conversation progresses.
* The importance of this interaction is high because it reveals a significant conflict between two agents who are supposed to be working together. The misunderstanding over the duck books sets off a chain reaction of hurt feelings, frustration, and defensiveness, which ultimately leads to a breakdown in their relationship. This kind of conflict can have long-term consequences for their partnership and overall success in their goals.

Parsed emotion: {'emotion': 'angry', 'importance': '8'}

Opinion about the conversation: Alice viewed Eliana as someone who was extremely knowledgeable and kind, but also overly attached and possessive of her interests, particularly when it came to

Conclusion: Eliana loves ducks